<div style="background-color:#000;"><img src="pqn.png"></img></div>

In [ ]:
import polars as pl
import yfinance as yf
import duckdb

### Retrieve and store financial data for further analysis

This code block downloads historical stock price data for NVIDIA (NVDA) from Yahoo Finance. It retrieves data from the beginning of 2023 to the end of the year 2023. The data is then converted into a Polars DataFrame, which is a high-performance data manipulation library similar to Pandas. A new column is added to the DataFrame to label the data with the stock symbol "NVDA".

In [ ]:
prices = yf.download("NVDA", start='2023-01-01', end='2024-01-01')

df = (
    pl
    .from_pandas(
        prices
        .reset_index()
    )
    .with_columns(
        [pl.lit("NVDA").alias("symbol")]
    )
)

The code uses Yahoo Finance to download stock prices for NVIDIA, covering the specified date range. The downloaded data is in Pandas format, which is then converted into a Polars DataFrame for efficient processing. After resetting the index to ensure the date is a regular column, a new column "symbol" is added with the value "NVDA" for easy identification of the stock in later analyses. This setup is particularly useful for managing and analyzing large datasets efficiently.

### Create a database and store the stock data for querying

This code initializes a connection to a DuckDB database called 'stocks.db' and creates a table named 'stocks'. If the table already exists, it will not create a new one. The data from our Polars DataFrame is stored in this table. This allows us to use SQL queries on the stock data, which is particularly helpful for complex data manipulations and analyses.

In [ ]:
con = duckdb.connect('stocks.db')
con.execute("""
    CREATE TABLE IF NOT EXISTS stocks AS SELECT * FROM df
""")

The connection to DuckDB is established, allowing for efficient SQL operations on data stored in-memory or on disk. The code checks if a table named 'stocks' exists in the database. If not, it creates it using the data from the Polars DataFrame. This integration of Polars and DuckDB enables the user to leverage SQL queries for data analysis, combining the strengths of both SQL and DataFrame manipulations.

### Perform SQL queries to extract insights from the data

Here, we execute SQL queries on the stored stock data to calculate the average closing price and find high-volume trading days. The first query calculates the average closing price for NVDA and the second query identifies the top 5 days with the highest trading volume. This approach highlights how SQL can be used to quickly summarize and explore financial data.

In [ ]:
pl.DataFrame(
    con.execute("""
    SELECT symbol, round(avg(Close), 2) as avg_close FROM stocks GROUP BY symbol
    """).fetchdf()
)

In [ ]:
pl.DataFrame(
    con.execute("""
    SELECT symbol, 
           date, 
           Volume,
           Close
    FROM stocks 
    ORDER BY Volume DESC 
    LIMIT 5
    """).fetchdf()
)

The SQL query calculates the average closing price of NVIDIA stock by grouping all data entries by the stock symbol and averaging the closing prices. The result is then rounded to two decimal places for clarity. The second query retrieves the top 5 trading days with the highest volume of trades. The results are converted into Polars DataFrames, allowing easy manipulation and visualization of the output. These queries provide a quick overview of key metrics, such as average price and trading volume peaks.

### Calculate rolling VWAP to track stock price trends

We use SQL to calculate the 20-day rolling Volume Weighted Average Price (VWAP) for NVIDIA stock. The VWAP is an important metric for traders as it provides insights into the average price a stock has traded at, factoring in volume. The rolling VWAP smooths out daily fluctuations to reveal longer-term trends.

In [ ]:
vwap_query = """
WITH daily_vwap AS (
    SELECT 
        "Date",
        symbol,
        SUM(Volume * Close) / SUM(Volume) as vwap
    FROM stocks
    GROUP BY "Date", symbol
),
rolling_vwap AS (
    SELECT 
        "Date",
        symbol,
        AVG(vwap) OVER (
            PARTITION BY symbol 
            ORDER BY "Date" 
            ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
        ) as rolling_20d_vwap
    FROM daily_vwap
)
SELECT * FROM rolling_vwap
ORDER BY symbol, "Date";
"""
pl.DataFrame(con.execute(vwap_query).fetchdf())

The code defines a SQL query that first calculates the daily VWAP by dividing the total value traded (Volume * Close) by the total volume for each day. Next, it computes a 20-day rolling average of these daily VWAPs to smooth short-term fluctuations and reveal longer-term price trends. The use of a window function in SQL allows the rolling average to be calculated efficiently. The output is then converted into a Polars DataFrame for further analysis or visualization. This process helps traders and analysts understand how the stock's price trends over time, adjusted for trading volume.

In [ ]:
con.close()

Closing the DuckDB connection ensures that all resources are freed and the database is properly closed. This is a good practice to prevent memory leaks and ensure data integrity after completing database operations. 

### Your next steps

Now try modifying the code to download data for a different stock symbol, such as AAPL or TSLA. You could also change the date range to analyze different periods. Experimenting with these parameters will help you understand how the code adapts to different data inputs and can provide new insights into other stocks' performance. This hands-on approach will deepen your understanding of data manipulation and analysis in quantitative finance.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.